Customer Segmentation & Product Insights for Online Retail
📌 Introduction : 
In this notebook, we perform an end-to-end analysis of customer behavior using an online retail dataset from Kaggle. 
The main goals are to identify customer segments through clustering and to analyze product purchasing patterns.
Using scalable tools like Dask and scikit-learn, we clean and preprocess the data, engineer key behavioral features (such as recency and average quantity),
and apply KMeans clustering to segment customers. 
We also identify the top-performing products by sales volume and visualize customer clusters using PCA for dimensionality reduction. 
This analysis supports more informed decisions in marketing, inventory, and customer relationship strategies.



We begin by installing the complete Dask library, which enables efficient, scalable operations on large datasets. This is especially helpful for loading and processing data that may not fit entirely into memory.

In [1]:
!pip install dask[complete]
import dask.dataframe as dd


In [2]:
import pandas as pd

We use Dask to load the dataset in parallelized chunks to avoid memory overflows. The `InvoiceNo` column is explicitly read as an object to preserve any alphanumeric invoice identifiers.

In [3]:
import dask.dataframe as dd
df = dd.read_csv("C:/Users/chahd/Desktop/Online Retail.csv", dtype={'InvoiceNo': 'object'})
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


Compute and print the total number of missing values in each column. This informs subsequent data cleaning steps, helping us decide which columns or rows need to be addressed or dropped.

In [20]:
 missing_values = df.isnull().sum().compute()
 print("Missing values per column:")
 print(missing_values)

Missing values per column:
InvoiceNo         0
StockCode         0
Description    1454
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID        0
Country           0
dtype: int64


We drop rows that have missing values in key columns: `CustomerID`, `Quantity`, and `StockCode`. These fields are critical for behavior analysis and clustering.

In [7]:
df_cleaned = df.dropna(subset=['CustomerID', 'Quantity', 'StockCode'])

Duplicate transactions are removed to prevent skewed customer metrics like purchase frequency or total spend.

In [9]:
df_cleaned = df_cleaned.drop_duplicates()

Data types are standardized for efficient computation and consistency. For example:
- `InvoiceDate` is converted to datetime for time-based calculations.
- `Quantity` is converted to float to ensure numerical accuracy.
- `StockCode` is cast to string to treat it as a categorical identifier

In [11]:
df_cleaned['InvoiceDate'] = dd.to_datetime(df_cleaned['InvoiceDate'])
df_cleaned['Quantity'] = df_cleaned['Quantity'].astype('float64')
df_cleaned['StockCode'] = df_cleaned['StockCode'].astype('str')

Negative quantities often represent product returns or corrections. For behavior modeling, we focus only on successful, positive-quantity transactions.

In [18]:
df_cleaned = df_cleaned[df_cleaned['Quantity'] > 0]

After preprocessing, the Dask DataFrame is computed into a standard Pandas DataFrame to enable more flexible operations like groupings and clustering.

In [20]:
cleaned_data=df_cleaned.compute()

We aggregate customer behavior data using:
- Total quantity purchased
- Date of last transaction (recency base)
- Total number of items (proxy for number of transactions or activity level)

In [22]:
df_customer_behavior = cleaned_data.groupby('CustomerID').agg({
    'Quantity': 'sum',
    'InvoiceDate': 'max',  
    'StockCode': 'count'  
}).reset_index()

In [24]:
df_customer_behavior['recency'] = (pd.to_datetime('today') - df_customer_behavior['InvoiceDate']).dt.days

In [26]:
df_customer_behavior['avg_quantity_per_transaction'] = df_customer_behavior['Quantity'] / df_customer_behavior['StockCode']

In [28]:
df_customer_behavior.head()

,CustomerID,Quantity,InvoiceDate,StockCode,recency,avg_quantity_per_transaction
0,12346,74215.0,2011-01-18 10:01:00,1,5194,74215.000000
1,12347,2458.0,2011-12-07 15:52:00,182,4871,13.505495
2,12348,2341.0,2011-09-25 13:13:00,31,4944,75.516129
3,12349,631.0,2011-11-21 09:51:00,73,4887,8.643836
4,12350,197.0,2011-02-02 16:01:00,17,5179,11.588235


We apply KMeans clustering to segment customers based on:
- Total quantity
- Recency
- Number of transactions
- Avg. quantity per transaction

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


features = ['Quantity', 'recency', 'StockCode', 'avg_quantity_per_transaction']
X = df_customer_behavior[features]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


kmeans = KMeans(n_clusters=5, random_state=42)
df_customer_behavior['cluster'] = kmeans.fit_predict(X_scaled)

print("Customer Behavior with Clusters:")
print(df_customer_behavior.head())

Customer Behavior with Clusters:
   CustomerID  Quantity         InvoiceDate  StockCode  recency  \
0       12346   74215.0 2011-01-18 10:01:00          1     5194   
1       12347    2458.0 2011-12-07 15:52:00        182     4871   
2       12348    2341.0 2011-09-25 13:13:00         31     4944   
3       12349     631.0 2011-11-21 09:51:00         73     4887   
4       12350     197.0 2011-02-02 16:01:00         17     5179   

   avg_quantity_per_transaction  cluster  
0                  74215.000000        2  
1                     13.505495        0  
2                     75.516129        0  
3                      8.643836        0  
4                     11.588235        3  


We extract the top customers from a specific cluster (e.g., Cluster 0), which is assumed to contain highly engaged or high-value clients.

In [32]:
high_engagement_cluster = 0
top_clients = df_customer_behavior[df_customer_behavior['cluster'] == high_engagement_cluster]
top_clients = top_clients.sort_values(by='Quantity', ascending=False)

print("Top High-Engagement Clients:")
print(top_clients.head())

Top High-Engagement Clients:
      CustomerID  Quantity         InvoiceDate  StockCode  recency  \
1690       14646  197491.0 2011-12-08 12:12:00       2080     4870   
3009       16446   80997.0 2011-12-09 09:15:00          3     4869   
1880       14911   80490.0 2011-12-08 15:54:00       5672     4870   
55         12415   77670.0 2011-11-15 14:22:00        716     4893   
3729       17450   69973.0 2011-12-01 13:29:00        336     4877   

      avg_quantity_per_transaction  cluster  
1690                     94.947596        0  
3009                  26999.000000        0  
1880                     14.190762        0  
55                      108.477654        0  
3729                    208.252976        0  


High Purchase Volume: Customers in Cluster 0 have significantly high total quantities purchased, often well above 70,000 units. This marks them as high-value clients.

Long Recency Values: The recency values (close to 4870 days) seem unusually high. This suggests a potential issue with the reference date used (possibly due to miscalculation or incorrect InvoiceDate parsing). This should be revisited to ensure accuracy.

Diverse Buying Patterns:

Customer 16446 has an extraordinarily high average quantity per transaction (~27,000), indicating bulk purchases or wholesale-like behavior.

Others have lower average quantities but still fall into the high-value bracket due to frequency or variety.

Business Insight: This cluster likely represents VIP customers or B2B clients who purchase in high volume. These customers should be prioritized for loyalty programs, personalized offers, or premium support.



We group the cleaned transactional data by `StockCode` to determine total quantity sold per product.

In [36]:
top_products = cleaned_data.groupby('StockCode').agg({
    'Quantity': 'sum'
}).reset_index()

Sort the product list in descending order to identify the most purchased items.

In [38]:
top_products = top_products.sort_values(by='Quantity', ascending=False)
print("Most Bought Products:")
print(top_products.head())

Most Bought Products:
     StockCode  Quantity
2470     23843   80995.0
2024     23166   78033.0
1112     22197   56898.0
2920     84077   54951.0
3404    85099B   48375.0


Top-Selling Products (StockCodes 23843, 23166, etc.) have sold extremely high volumes, ranging from ~48K to ~81K units.

The data suggests strong SKU-level concentration: A small number of products account for a significant portion of overall sales.

These products may be:

Staple or best-selling items (e.g., popular gifts or household essentials),

High-margin stock frequently promoted or bundled,

Or possibly used in seasonal promotions, which should be cross-verified with time-based sales analysis.

📌 Business Implications:
These products are core revenue drivers and should be kept in inventory with prioritized supply chain planning.

Summary:
Cluster 0 contains high-volume, highly engaged customers—potentially corporate or wholesale buyers.

The most bought products are likely strategic stock items that deserve attention in both operations and marketing.